# FIT5196 Task 4 in Assessment 2
#### Student Name: Chuangfu Xie
#### Student ID: 27771539

Date: 13/05/2018

Version: 1.0

Environment: Python 3.6.4
Packages list:
1. Pandas 0.22.0  
2. numpy 1.14.0
4. sklearn 0.19.1

In [ ]:
import sys
print (sys.version)

## 1.  Import libraries 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor

## 2. Inspection

In [ ]:
df = pd.read_csv('./dataset4_with_outliers.csv')
# have a peek at the dataset
df.head()

### 2.1 Feature Selection

First, we use `check_all_corr()` function to sift those columns which correlation coefficient are under the theshold, and then return the columns need to drop and all potentially related columns:

In [ ]:
def check_all_corr(df, theshold=0.5):
    col_list=df.corr().columns.tolist()
    pairs_dict=dict()
    for c in col_list:
        for r in col_list:
            corr_value = df.corr().loc[r,c]
            if corr_value > theshold and corr_value!=1:
                if c not in pairs_dict:
                    pairs_dict[c]=dict()
                else:
                    pairs_dict[c][r] = corr_value #save pair
    # remove columns which pass the theshold
    drop_list = df.columns.tolist()
    for c in pairs_dict:
        for r in pairs_dict[c]:
            if r in drop_list:
                drop_list.remove(r)
        if c in pairs_dict:
            try:
                drop_list.remove(c)
            except:
                pass
    return drop_list, pairs_dict

In [ ]:
drop_list, corr_results = check_all_corr(df)
print(drop_list)

Now we have all candidate columns are potentially having no relationship with other columns. However, we can not just simply drop all of them. Let's have a closed look at these columns.  
Accroding to instructions, we can find that:
1. Data in <font color='blue'>**id**</font> are just identifier, we can drop this.
2. Each record can have different value in <font color='blue'>**date**</font>, we can drop this.
3. <font color='blue'>**zipcode**</font>, <font color='blue'>**lat**</font> and <font color='blue'>**lon**</font> are used to locate the building, we can drop this.
4. <font color='blue'>**waterfront**</font> are just feature of the building, but may having relationship with <font color='blue'>**view**</font>, though the value of the view takes might be subjective to some extent. we keep this.
5. From task 3 we knows <font color='blue'>**sqft_basement**</font>, <font color='blue'>**sqft_living**</font> and <font color='blue'>**sqft_above**</font> obey to a formula. However, <font color='blue'>**sqft_lot**</font> can be outlier. So, we keep <font color='blue'>**sqft_lot**</font>.
6. <font color='blue'>**yr_renovated**</font> are just history, where a renovation happens, the value will be non-zero. On the other hand, <font color='blue'>**condition**</font> may propriately describe at what degree the building is maintained. Thus, we just keep the <font color='blue'>**condition**</font>.

In [ ]:
drop_list = ['id', 'date', 'sqft_basement', 'yr_renovated', 'zipcode', 'lat', 'long']
df_corr = df.drop(columns=drop_list)
df_corr.describe()

### 2.2 Columns contains the most number of outliers

In [ ]:
ax, lines = df_corr.boxplot(figsize=(20,10), return_type='both')
all_outliers = dict()
for col, each in zip(df_corr.columns.tolist(), lines['fliers']):
    # collect all outliers
    all_outliers[col] = each.get_ydata().tolist()

In [ ]:
n_outliers = dict()
for col in all_outliers:
    n_outliers[col] = len(all_outliers[col])
n_outliers

Now we can find that the column which contains the most significant number of outliers is <font color='blue'>**sqft_lot**</font>: 1111 potential outliers.

### 2.3 LOF –– Local Outlier Factor

As there are 12 columns to consider, we can not simply identify outliers by Boxplot. Since we know all the correlation coefficient of these columns, however, outlier should be identify by the pattern which most records are grouping in a cluster-like. To this extent, we employ LOF technique.

As LOF share some fundamental concepts with density-based clustering approaches, by reeferencing to sklearn documentation:
> "... the anomaly score depends on **how isolated the object is with respect to the surrounding neighborhood**. More precisely, **locality is given by k-nearest neighbors**, whose distance is used to **estimate the local density**. By comparing the local density of a sample to the local densities of its neighbors, one can identify samples that have a substantially **lower density than their neighbors**, These are considered **outliers**."

First, we need to find the **k**, as we can't visualise the clustering in the dataset, we can use a simple method to estimate k:

In [ ]:
k = np.sqrt(len(df.index))
print(k)

As k is usually odd, let's assume that k is 100. Since the most number of potential outliers are occurs in <font color='blue'>**sqft_lot**</font> , we use it to calculate the contamination ratio, i.e the proportion of outliers in the data set:

In [ ]:
model = LocalOutlierFactor(100, p=13, contamination=1111/len(df.index))
df_corr['outliers'] = model.fit_predict(df_corr)

Now we have all the outliers predicted by LOF model. let's check on these outlier with price per square footage:

In [ ]:
df_outlier = df_corr[df_corr.outliers == -1]
df_outlier = np.round(df_outlier.price/df_outlier.sqft_lot)
df_outlier.describe()

In [ ]:
df_no = df_corr[df_corr.outliers == 1]
df_no = np.round(df_no.price/df_no.sqft_lot)
df_no.describe()

Here we can find data distribution with data in outliers against others:
1. Data in outliers: mean: 18.25; median: 13.00; std:37.2
2. Data without outliers: mean: 95.10; median: 64; std: 94.45

It seems that data in outliers are having extreme low price with mean of 18.25! Let's check in another way:

In [ ]:
df_outlier.plot()

In [ ]:
df_no.plot()

It seems that most of the outliers has been identified.  
Then we can remove these outliers:

In [ ]:
df = df[df_corr['outliers']==1]

## 3. Export CSV 

In [ ]:
df.to_csv('./dataset4_solution.csv',index=False)

### Reference

* Thirumuruganathan S. (May 17, 2010). *A Detailed Introduction to K-Nearest Neighbor (KNN) Algorithm* Retrieve from:[https://saravananthirumuruganathan.wordpress.com/2010/05/17/a-detailed-introduction-to-k-nearest-neighbor-knn-algorithm/](https://saravananthirumuruganathan.wordpress.com/2010/05/17/a-detailed-introduction-to-k-nearest-neighbor-knn-algorithm/)